# Imports

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
!export PYSPARK_PYTHON=python
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 338 kB in 2s (136 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
18 packages can be upgraded. Run 'apt list --upgradable' to see them.
openjdk-8-jdk-headless is already the n

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import math
from datetime import datetime

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Inicialización de los RDD

In [ ]:
id1="17SvnFyBNyY018rxrRnOLGp9YaZfeRknp"
downloaded1 = drive.CreateFile({'id': id1})
downloaded1.GetContentFile('googleplaystore.csv')

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

sqlContext = SQLContext(sc)

apps_df = sqlContext.read.csv('googleplaystore.csv',
                              header=True,
                              inferSchema=True,
                              mode='DROPMALFORMED') \
                              .select("App", "Size", "Android Ver")
apps_rdd = apps_df.rdd

# Pregunta #18
[*ir al notebook general*](https://colab.research.google.com/drive/1wWj79hKJTtaauQYJndtlhfQ8KEiuKe8Q?usp=drive_link)

> *Calcule el tamaño promedio de las aplicaciones por versión de Android, sin tener en cuenta las aplicaciones que varían en tamaño según dispositivo*

**Respuesta:** *ver salida, resultados en Mb*

**Notas:**
- El `size_to_float` me convierte todos los tamaños bien: Convierte los que tienen M a megas, los que tienen K a kilos, y los que no sean un número (Varies with device y un par de filas mal formateadas) me los pone como nan
- Aún con eso sobrevivió una categoría mal formateada ('NaN), por eso el chequeo extra
- Las categorías no las toqué (ademas de la 'nan' que estaba mal), porque había preguntado al respecto por un ejercicio de pandas que también era con las categorías y me dijeron que, aparte de las líneas malas, todas las categorías las puedo tomar como "correctas"
- Trabajé todo en Mb proque me parece más acorde a la escala que tienen los tamaños de las apps (prefiero un par de apps con 0.5 Mb a la mayoría con 20000 Kb)
- El primer map y reduceByKey me quitan las apps duplicadas. Hago básicamente lo mismo (map a dupla nombre - campos que quiero y reduceByKey de agarrar uno de los 2) en el resto de los ejercicios que usan el dataset de apps. No tengo ningún criterio para seleccionar a uno de los duplicados, así que agarro uno cualquiera

In [ ]:
def size_to_float(sz):
  try:
    char = sz[-1]
    sz = sz[:-1]
    return float(sz) if char == 'M' else float(sz)/1024
  except:
    return math.nan

ans18 = apps_rdd \
  .map(lambda x: (x[0], (x[1], x[2]))) \
  .reduceByKey(lambda x,y: x) \
  .map(lambda x: (x[1][1], (size_to_float(x[1][0]), 1))) \
  .filter(lambda x: not (math.isnan(x[1][0]) or 'NaN' in x[0])) \
  .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1])) \
  .map(lambda x: (x[0], x[1][0]/x[1][1])) \
  .take(33)

# para la salida
red = '\033[91m'
yellow = '\033[93m'
white = '\033[97m'

print(f'{red}============================{white}')
for cat in ans18:
    print(f"{yellow}{cat[0]}: {white}{cat[1]:.2f} Mb")
print(f'{red}============================{white}')

4.0.3 and up: 21.22 Mb
4.2 and up: 21.15 Mb
4.4 and up: 25.27 Mb
2.3 and up: 20.26 Mb
3.0 and up: 13.73 Mb
4.1 and up: 24.28 Mb
4.0 and up: 16.49 Mb
2.2 and up: 8.22 Mb
5.0 and up: 23.31 Mb
6.0 and up: 19.46 Mb
1.6 and up: 3.11 Mb
2.1 and up: 5.58 Mb
5.1 and up: 24.06 Mb
1.5 and up: 5.07 Mb
7.0 and up: 24.31 Mb
4.3 and up: 21.92 Mb
4.0.3 - 7.1.1: 6.60 Mb
2.0 and up: 6.25 Mb
2.3.3 and up: 18.66 Mb
Varies with device: 19.91 Mb
3.2 and up: 9.86 Mb
4.4W and up: 14.13 Mb
7.1 and up: 32.00 Mb
7.0 - 7.1.1: 7.10 Mb
8.0 and up: 20.08 Mb
5.0 - 8.0: 23.80 Mb
3.1 and up: 13.41 Mb
2.0.1 and up: 22.22 Mb
4.1 - 7.1.1: 7.90 Mb
5.0 - 6.0: 11.00 Mb
1.0 and up: 3.85 Mb
2.2 - 7.1.1: 5.10 Mb
5.0 - 7.1.1: 8.40 Mb
